### 쿠팡에 로그인하여 링크의 상품을 장바구니에 추가
### # 같은 버전의 Chrome 과 chromedriver 가 필요합니다.
### # chromedriver.exe 가 현재 파일과 같은 위치에 존재해야 합니다.

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
def add_to_cart_coupang(coup_id, coup_pw, item) :
    # chromedriver.exe 파일 위치에 따라 변경해주어야 합니다.
    driver = webdriver.Chrome(u'chromedriver')

    # 로그인
    driver.get("https://login.coupang.com/login/login.pang")
    # ID
    elem_login_id = driver.find_element_by_id("login-email-input")
    elem_login_id.clear()
    elem_login_id.send_keys(coup_id)
    # PW
    elem_login_pw = driver.find_element_by_id("login-password-input")
    elem_login_pw.clear()
    elem_login_pw.send_keys(coup_pw)
    
    xpath = "//button[text()='로그인']"
    driver.find_element_by_xpath(xpath).click()
    # 로그인 완료될때까지 대기
    element = WebDriverWait(driver, 5).until(
        EC.text_to_be_present_in_element(
            (By.XPATH, "//span[text()='로그아웃']"),
            '로그아웃'
        )
    )

    # link 의 상품 장바구니 담기
    driver.get(item['링크'])
    # 상품정보 페이지 로딩 완료될때까지 대기
    element = WebDriverWait(driver, 5).until(
        EC.text_to_be_present_in_element(
            (By.NAME, 'detail'),
            '상품상세'
        )
    )
    
    xpath = '//button[@class="prod-cart-btn"]'
    driver.find_element_by_xpath(xpath).click()
    # 장바구니에 담길때까지 대기
    element = WebDriverWait(driver, 5).until(
        EC.text_to_be_present_in_element(
            (By.CLASS_NAME, 'prod-order-notifier-content'),
            '상품이 장바구니에 담겼습니다.'
        )
    )
    
    # 장바구니로 이동
    xpath = '//a[@class="clearFix mycart-preview-module"]'
    driver.find_element_by_xpath(xpath).click()
    # 장바구니 페이지 로딩 완료될때까지 대기
    element = WebDriverWait(driver, 5).until(
        EC.text_to_be_present_in_element(
            (By.ID, 'th-product-box'),
            '상품정보'
        )
    )

    if driver.find_element_by_xpath("//a[contains(text(), '" + item['상품명'] + "')]") is not None :
        print(item['상품명'] + ' 이 장바구니에 담겼습니다.')
    else :
        print(item['상품명'] + ' 이 장바구니에 담기지 않았습니다.')

### 쿠팡에서 최저가 검색후 {상품명, 원가, 할인율, 판매가, 비고, 링크} 를 딕셔너리로 반환

In [3]:
import re
import requests
from bs4 import BeautifulSoup as bs

In [4]:
# pname = 상품명
def get_cheapest_item_coupang(pname) :
    base_url = 'https://www.coupang.com/np/search'

    params = {
        'q': pname,
        'channel': 'user',
        'component': None
    }

    # 미포함시 접근 불가
    headers = {
        'Referer': 'https://www.coupang.com/',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4220.0 Safari/537.36'
    }
    
    resp = requests.get(base_url, params=params, headers=headers)
    soup = bs(resp.text)
    items = soup.select('li.search-product') # 상품정보 1개를 li.search-product 가 감싸고 있음.
    
    if soup.select('p.no-hit') : # 검색된 상품이 없을시
        return dict( # 임시 상품 반환.
            상품명 = pname,
            원가 = 0,
            할인율 = '0%',
            판매가 = 0,
            비고 = '(상품이 존재하지 않습니다)',
            링크 = None
        )
    
    # 두번째 상품부터 로켓프레시가 아니고, 검색 키워드를 포함하는 첫 상품을 검색.
    # 로켓프레시는 장바구니 담기가 안되어 제외시켜야 함.
    idxFirstItem = 1 # 첫번째 상품은 보통 광고인데 결과가 정확하지 않은 경우가 많아 제외 (사과 -> 사과식초, 배추 -> 배추김치)
    num_items = len(items)
    while idxFirstItem < num_items :
        tmpItem = items[idxFirstItem].select_one('span.badge.rocket > img') # 로켓프레시, 로켓배송 등이 가지고 있는 태그
        if pname in items[idxFirstItem].find('div', class_='name').text : # 검색 키워드를 포함하고
            if tmpItem is None : # 일반 상품(일반 상품은 장바구니 담기 가능)이면 사용.
                break
            else : # 일반 상품이 아니면(로켓프레시나 로켓배송 등)
                if 'rocket-fresh' not in tmpItem['src'] :
                    break # 로켓프레시가 아니면 사용
                else :
                    pass # 로켓프레시면 다음 상품 확인.
        idxFirstItem += 1
    
    item = items[idxFirstItem]
    info = {}
    
    # 상품명
    info['상품명'] = item.find('div', class_='name').text
     
    # 할인율, 원가, 판매가
    # span.instant-discount-rate 가 존재할 경우, 안에 할인율이 들어있음.
    objDiscountRate = item.find('span', class_='instant-discount-rate')
    price = int(re.sub("\D", "", item.find('strong', class_='price-value').text)) # 판매가
    if objDiscountRate is not None :
        info['할인율'] = objDiscountRate.text
        info['원가'] = int(re.sub("\D", "", item.find('del', class_='base-price').text))
        info['판매가'] = price
    else : # 할인율 정보 없을시, 0% (원가 = 판매가)
        info['할인율'] = '0%'
        info['원가'] = info['판매가'] = price
    
    # 상세정보
    # span.unit-price 가 존재할 경우, (100g당 400원) 형식으로 들어있음.
    objSpec = item.find('span', class_='unit-price')
    if objSpec is not None :
        info['비고'] = objSpec.text.strip()
    else : # 상세 정보 없을시 배송 정보로 대체
        info['비고'] = item.find('span', class_='arrival-info').text.strip()
    
    # 링크
    info['링크'] = 'https://www.coupang.com' + item.find('a', class_='search-product-link')['href']
    
    return info

### 쿠팡 최저가 검색 실행 예시

In [5]:
print(get_cheapest_item_coupang('대파'))
print()
print(get_cheapest_item_coupang('고추장'))
print()
print(get_cheapest_item_coupang('마늘'))
print()
print(get_cheapest_item_coupang('라면'))
print()
print(get_cheapest_item_coupang('생닭'))
print()
print(get_cheapest_item_coupang('없는상품이니까검색하지마세요'))

{'상품명': '빛고을장터 국내산 무농약 대파 3kg 1kg내외, 1봉, 01_무농약 대파 (특/1kg)', '할인율': '14%', '원가': 3500, '판매가': 3000, '비고': '모레(금) 8/7   도착 예정', '링크': 'https://www.coupang.com/vp/products/1522323055?itemId=2612173783&vendorItemId=3628600382'}

{'상품명': '청정원순창 100% 현미 태양초 찰고추장, 2kg, 1개', '할인율': '56%', '원가': 24600, '판매가': 10780, '비고': '(100g당 539원)', '링크': 'https://www.coupang.com/vp/products/96414?itemId=1015299&vendorItemId=3001106979'}

{'상품명': '청풍명가 2020년 명품 햇 다진마늘, 1개, 500g', '할인율': '28%', '원가': 5900, '판매가': 4200, '비고': '모레(금) 8/7   도착 예정', '링크': 'https://www.coupang.com/vp/products/1309129255?itemId=2325566891&vendorItemId=70322210043&sourceType=srp_product_ads'}

{'상품명': '농심 신라면 120g x 5p + 안성탕면 125g x 5p + 얼큰 너구리 120g x 5p + 짜파게티 140g x 5p, 1세트', '할인율': '0%', '원가': 14200, '판매가': 14200, '비고': '내일(목) 8/6   도착 보장', '링크': 'https://www.coupang.com/vp/products/1910849030?itemId=3244307968&vendorItemId=70916247951'}

{'상품명': '가나안식품 생닭 대 2마리, 1세트, 2.4kg', '할인율': '0%', '원가': 9990, '판매가': 9990, '비고': '모레(금) 

### 쿠팡 장바구니 담기 실행 예시

In [6]:
import base64

coup_id = base64.b64decode(b'a3l0OTYwMUBuYXZlci5jb20=').decode("UTF-8")
coup_pw = base64.b64decode(b'aHQhQGd0MzQ=').decode("UTF-8")

item = get_cheapest_item_coupang('당근')
if item['링크'] is not None : add_to_cart_coupang(coup_id, coup_pw, item)

item = get_cheapest_item_coupang('시금치')
if item['링크'] is not None : add_to_cart_coupang(coup_id, coup_pw, item)
    
item = get_cheapest_item_coupang('양배추')
if item['링크'] is not None : add_to_cart_coupang(coup_id, coup_pw, item)

세척당근 특품, 1박스, 01_세척당근(특) 5kg 이 장바구니에 담겼습니다.
(신선) 시금치 spinach 1단 국내산, 단일상품 이 장바구니에 담겼습니다.
빛고을장터 국내산 양배추 1통 (2~3kg내외), 국내산 양배추 (2~3kg내외) 이 장바구니에 담겼습니다.
